[View in Colaboratory](https://colab.research.google.com/github/x110/MyFastaiNotebooks/blob/master/lang_model_arxiv.ipynb)

# Sentiment Classification


## Introduction

### Objective
The sentiment classification task consists of predicting the polarity (positive or negative) of a given text. 


###  Dataset
The [large moview view dataset](http://files.fast.ai/data/aclImdb.tgz) contains a collection of 50,000 reviews from IMDB. The datset contains an even number of positive and negative reviews. The authors considered only highly polarized reviews. A negative review has a score $\leq 4$ out of 10, a positive review has a score $\geq 7$ out of 10. Nuteral reviews are not included in this dataset, 
The dataset we are using is the IMDb movie review dataset. "This is a dataset of binary sentiment classification. It contains a set of 25,000 higly polar movie reviews for training, and 25,000 for testing.  

### Solution Strategy
Before we try to classify sentiment, we will try to create a language model; that is, a model that can predict the next word in a sentence. Why? Because our model first needs to understand the structure of English, before we can expect it to recognize positive or negative sentiment. 

## Prepare virtual machine on google colab


1. Connect to google drive
2. Install dependencies

In [1]:
!ls

datalab


In [2]:
## Connect to google drive for storage 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
#Install dependencies
!pip install https://github.com/fastai/fastai/archive/master.zip
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install kaggle

    100% |████████████████████████████████| 90.0MB 516kB/s 
    100% |████████████████████████████████| 1.5MB 5.1MB/s 
    100% |████████████████████████████████| 112kB 8.8MB/s 
    100% |████████████████████████████████| 184kB 9.8MB/s 
    100% |████████████████████████████████| 3.4MB 10.1MB/s 


    63% |████████████████████▏           | 312.9MB 33.3MB/s eta 0:00:06

    100% |████████████████████████████████| 496.4MB 29kB/s 
    100% |████████████████████████████████| 51kB 11.5MB/s 
    100% |████████████████████████████████| 61kB 20.5MB/s 
    100% |████████████████████████████████| 51kB 12.2MB/s 
    100% |████████████████████████████████| 2.2MB 10.3MB/s 
    100% |████████████████████████████████| 13.7MB 3.6MB/s 
    100% |████████████████████████████████| 92kB 14.9MB/s 
    100% |████████████████████████████████| 112kB 20.1MB/s 
    100% |████████████████████████████████| 71kB 20.0MB/s 


    100% |████████████████████████████████| 81kB 16.7MB/s 
  Running setup.py bdist_wheel for fastai ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-dzyaompj/wheels/64/3d/9f/d12a217aa2531321c5b9ae96288fcae2687d3b744376e8f94f
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

 - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Running setup.py bdist_wheel for feather-format ... - done
  Stored in directory: /content/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
  Running setup.py bdist_wheel for pandas-summary ... - done
  Stored in directory: /content/.cache/pip/wheels/86/a9/6b/36b727a9ec687dac08bd3d0501cddd7b1a223943513eb04a03
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built fastai bcolz feather-format pandas-summary torchtext
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
torchvision 0.2.1 has 

    100% |████████████████████████████████| 592.3MB 16.4MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: torch 0.3.1
    Uninstalling torch-0.3.1:
      Successfully uninstalled torch-0.3.1
    100% |████████████████████████████████| 2.0MB 8.1MB/s 
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/61/d6/e6/38fc0e2316f49b91aafdaf9a8d7756a9906b2ad7e501c1e165
Successfully built kaggle
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


In [4]:
#install More dependencies
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
!apt-get install p7zip-full

    100% |████████████████████████████████| 5.6MB 4.3MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.1MB 5.2MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [5]:
!mkdir .kaggle
!cp drive/kaggle.json /content/.kaggle/
!chmod 600 /content/.kaggle/kaggle.json

cp: cannot stat 'drive/kaggle.json': No such file or directory
chmod: cannot access '/content/.kaggle/kaggle.json': No such file or directory


In [6]:
!pip install spacy

    100% |████████████████████████████████| 17.6MB 1.7MB/s 
    100% |████████████████████████████████| 92kB 20.2MB/s 
    100% |████████████████████████████████| 1.2MB 14.8MB/s 
    100% |████████████████████████████████| 51kB 19.7MB/s 
    100% |████████████████████████████████| 194kB 22.4MB/s 
    100% |████████████████████████████████| 153kB 23.5MB/s 
    100% |████████████████████████████████| 604kB 19.8MB/s 
    100% |████████████████████████████████| 389kB 23.7MB/s 
    100% |████████████████████████████████| 143kB 20.4MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / -

 \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thinc ... - \ | / -

 \ | / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


## Downloads

### Download the weights of the trained language model

In [7]:
#already in drive

### Download data

In [8]:
!wget http://files.fast.ai/data/aclImdb.tgz

--2018-07-01 05:02:18--  http://files.fast.ai/data/aclImdb.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145982645 (139M) [text/plain]
Saving to: ‘aclImdb.tgz’

aclImdb.tgz         100%[===================>] 139.22M   106MB/s    in 1.3s    

2018-07-01 05:02:20 (106 MB/s) - ‘aclImdb.tgz’ saved [145982645/145982645]



In [9]:
!mkdir data

In [10]:
!tar -xzf aclImdb.tgz	

In [11]:
!mv aclImdb	data/

## Imports

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

## Language modeling
### Data

In [13]:
!ls

aclImdb.tgz  data  datalab  drive


In [14]:
PATH = 'data/aclImdb/'
TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

imdbEr.txt  imdb.vocab  README  test/  train/


In [15]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '1000_0.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt']

In [16]:
review=!cat {TRN}{trn_files[6]}
review[0] 

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out 

In [17]:
!find {TRN} -name "*.txt" | xargs cat| wc -w

17486581


In [18]:
!find {VAL} -name "*.txt" | xargs cat | wc -w

5686719


In [19]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 49.3MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [20]:
spacy_tok=spacy.load('en')

In [21]:
spacy_tok

In [22]:
' '.join(str(v) for v in spacy_tok(review[0]))

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop - socky fung - ku , but what I got instead was a comedy . So , it was n't quite was I was expecting , but I really liked it anyway ! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them ! ! I was laughing my ass off . I mean , the cops were just so bad ! And when I say bad , I mean The Shield Vic Macky bad . But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose ... man , oh man . What can you say about that hottie . She was great and put those other actresses to shame . She should work more often ! ! ! ! ! I also really liked the fight scene outside of the building . That was done really well . Lots of fighting and people getting their heads banged up . FUN ! Last , but not least Joe Estevez and William Smith were great as the ... well , I was n't sure what they were , but they see

In [23]:
TEXT=data.Field(lower=True,tokenize='spacy')#spacy_tok)#https://github.com/fastai/fastai/issues/213

In [24]:
type(list(spacy_tok(review[0]))[0])

spacy.tokens.token.Token

In [25]:
bs=64;bptt=70

In [26]:
FILES=dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)

In [27]:
%time md = LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=10)

CPU times: user 13min 59s, sys: 6.47 s, total: 14min 6s
Wall time: 14min 8s


In [28]:
md.nt

37392

In [29]:
!ls data/aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [30]:
!mkdir {PATH}models

pickle.dump(TEXT,open(f'{PATH}models/TEXT.pkl','wb'))

In [31]:
len(md.trn_dl),md.nt,len(md.trn_ds),len(md.trn_ds[0].text)

(4583, 37392, 1, 20540756)

In [42]:
!ls

aclImdb.tgz  data  datalab  drive


This is the start of the mapping from integer IDs to unique tokens

In [32]:
#'itoi':'string to int'
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [33]:
TEXT.vocab.stoi['the']

2

Note that in a languageModelData object there is only one item in each dataset: all words of the text joined together

In [34]:
md.trn_ds[0].text[:12]

['this',
 'movie',
 'is',
 'a',
 'must',
 'see',
 '.',
 'it',
 'shows',
 'poverty',
 'with',
 'a']

torchtext will handle turning these words into integer IDs for us automatically

In [35]:
TEXT.numericalize([md.trn_ds[0].text[0:12]])

Variable containing:
   13
   23
    9
    6
  220
   79
    4
   11
  288
 3268
   21
    6
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

Our language model object will create batches with 64 columns ,and varying sequence lengths of around 70 tokens.

Each batch also contains the exact sane data as labels,but one word later than the text since we are trying to predict the next words. the labels are flatten into a 1d array.

In [36]:
next(iter(md.trn_dl))

(Variable containing:
     13      8     31  ...      47     27    726
     23      2    604  ...      26     13     81
      9      0    176  ...      22    815    683
         ...            ⋱           ...         
   1115     61     32  ...       8     28   5992
    142     61     29  ...     580   4380  14795
    721     61    619  ...     270  24622      3
 [torch.cuda.LongTensor of size 68x64 (GPU 0)], Variable containing:
     23
      2
    604
   ⋮   
     99
     30
   2045
 [torch.cuda.LongTensor of size 4352 (GPU 0)])

## Train



In [37]:
em_sz=200#size of each embedding vector
nh=500 #number of hidden activation per layer
nl = 3 #number of layers

Researchers have found that large amounts of momentum dont work well with these kinds of RNN models, so we create a versio of adam optimizerwith less momentum than its default of .9

In [38]:
opt_fn = partial(optim.Adam,betas=(.7,.99))

fastai uses a variant of the state of the art AWD LSTM Language model developed by Stephen Merity. A key feature of that model is that it provides excellent regularization through dropout. 

The other parameters (alpha, beta, clip) should not generally need tuning.

In [39]:
learner = md.get_model(opt_fn,em_sz,nh,nl,
                      dropouti=.05, dropout = .05, wdrop=.1, dropoute=.02, dropouth=.05)
learner.reg_fn = partial(seq2seq_reg,alpha=2,beta=1)
learner.clip=.3

In [40]:
learner

SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(37392, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(37392, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=200, out_features=37392)
    (dropout): LockedDropout(
    )
  )
)

In [0]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      5.117613   5.003315  
 17%|█▋        | 781/4583 [03:47<18:27,  3.43it/s, loss=5.05]

    1      4.779484   4.641753  


    2      4.662469   4.551414  


    3      4.660255   4.524417  


    4      4.565983   4.443225  


    5      4.504306   4.386119  
 50%|█████     | 2292/4583 [11:14<11:14,  3.40it/s, loss=4.46]

In [0]:
learner.save_encoder("adam1_enc")

In [0]:
!cp data/aclImdb/models/adam1_enc.h5 drive

In [43]:
!cp drive/adam1_enc.h5 data/aclImdb/models/adam1_enc.h5

In [44]:

learner.load_encoder("adam1_enc")

In [45]:
#learner.load_cycle('adam1_enc',2)

In [49]:
%time learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      4.491605   4.358806  
 17%|█▋        | 781/4583 [03:44<18:15,  3.47it/s, loss=4.46]

    1      4.479334   4.347762  


    2      4.467662   4.329104  


    3      4.440199   4.309714  


    4      4.422475   4.291357  


...

    6      4.350979   4.250853  


    7      4.348734   4.239382  


    8      4.309778   4.230081  


    9      4.302973   4.22841   

CPU times: user 1h 25min 33s, sys: 2h 35min 55s, total: 4h 1min 28s
Wall time: 4h 1min 47s


[array([4.22841])]

In [50]:
learner.save_encoder('adam3_10_enc')
!cp data/aclImdb/models/adam3_10_enc.h5 drive

In [0]:
%time learner.fit(3e-3, 1, wds=1e-6, cycle_len=5)

epoch      trn_loss   val_loss   
    0      4.474314   4.332007  
 17%|█▋        | 781/4583 [03:43<18:07,  3.50it/s, loss=4.45]

    1      4.433271   4.303378  


    2      4.387978   4.267996  
 71%|███████   | 3265/4583 [15:34<06:17,  3.49it/s, loss=4.3]

In [0]:
learner.save_encoder('adam3_10_enc2')
!cp data/aclImdb/models/adam3_10_enc2.h5 drive

In [0]:
math.exp(4.165)

In [0]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [0]:
#uncomment to refresh google drive
!google-drive-ocamlfuse -cc

Clearing cache...done
